In [1]:
import os       # Os meeans Open System
import json
import pandas as pd
import traceback

In [10]:
from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv, find_dotenv
#find_dotenv()
load_dotenv("c:/Users/andre/mcqgen/.env")

True

In [11]:
KEY=os.getenv("OPENAI_API_KEY")

In [12]:
#print(KEY)

sk-proj-fkYjBcxORYfaknozEix3T3BlbkFJfmnNBoTtK7jjZd8z1Q32


In [ ]:
llm = ChatOpenAI(open_ai_key=KEY, model_name="gpt-3.5-turbo", temperature = 0.5)